Make the system ready

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
# --- Download and decompress the file ---
!wget -q "https://ftp.ncbi.nlm.nih.gov/geo/series/GSE194nnn/GSE194122/suppl/GSE194122_openproblems_neurips2021_cite_BMMC_processed.h5ad.gz" -O data.h5ad.gz
!gunzip -f data.h5ad.gz  # creates data.h5ad

In [ ]:
# --- 0) Install deps (Colab usually already has these, but safe) ---
!pip -q install anndata scanpy scipy pandas numpy

In [ ]:
# Colab: enable R cells via rpy2, then install Seurat & friends from CRAN
!pip -q install rpy2==3.5.11

# Start an R session and install packages
import os
os.environ["R_HOME"] = "/usr/lib/R"
os.environ["R_USER"] = "/root"

# The next cell uses %%R to run R code
# Load the R cell magic into Colab
%load_ext rpy2.ipython

In [ ]:
import os
import gzip
import numpy as np
import pandas as pd
import scanpy as sc
import scipy.sparse as sp
from scipy.io import mmwrite

preparing the dataset

In [ ]:
# 1) Paths / config
H5AD_PATH = "data.h5ad"
OUT_DIR = "/content/drive/MyDrive/Datasets/GSE194122/seurat/RNA_10x"  # <- change if you want
os.makedirs(OUT_DIR, exist_ok=True)

# Which layer to use as raw RNA counts:
# - Many datasets store raw counts in adata.layers["counts"]
# - Sometimes counts are in adata.X
PREFERRED_COUNT_LAYERS = ["counts", "raw_counts", "count", "RNA_counts"]


# 2) Load h5ad
adata = sc.read_h5ad(H5AD_PATH)
print("Loaded:", adata)
print("X:", type(adata.X), "shape:", adata.shape)
print("layers:", list(adata.layers.keys())[:10], "...")

# 3) Pick the RNA count matrix (non-multimodal)
def pick_counts_matrix(adata, preferred_layers):
    # 1) Prefer a counts layer if present
    for k in preferred_layers:
        if k in adata.layers:
            print(f"Using adata.layers['{k}'] as RNA counts.")
            return adata.layers[k], f"layers[{k}]"

    # 2) If adata.raw exists, it often stores unnormalized counts
    if adata.raw is not None and adata.raw.X is not None:
        # raw can be very large; still valid
        print("Using adata.raw.X as RNA counts (fallback).")
        return adata.raw.X, "raw.X"

    # 3) Fallback to adata.X
    print("Using adata.X as RNA counts (fallback).")
    return adata.X, "X"

X_counts, counts_source = pick_counts_matrix(adata, PREFERRED_COUNT_LAYERS)

# Ensure sparse CSR/CSC for writing
if not sp.issparse(X_counts):
    X_counts = sp.csr_matrix(X_counts)

# Safety: Seurat expects integer-ish counts for CreateSeuratObject
# If your matrix is float but represents counts, round safely.
# (If it's actually normalized/log, you *should not* round; but this dataset is typically counts.)
if X_counts.dtype.kind in ("f", "c"):
    # Check if values look integer-like
    sample = X_counts.data[:10000] if X_counts.nnz > 0 else np.array([0.0])
    frac = np.mean(np.abs(sample - np.round(sample)) > 1e-6) if sample.size else 0.0
    if frac < 0.01:
        print("Counts matrix is float but looks integer-like; rounding to int.")
        X_counts.data = np.rint(X_counts.data).astype(np.int64)
    else:
        print("WARNING: matrix has many non-integer values. It may be normalized/log.")
        print("Proceeding without rounding; Seurat will treat as data, not raw counts.")

# Convert to CSC for MatrixMarket (more standard for gene x cell writing)
X_counts = X_counts.tocsc()


# 4) Decide gene names and barcodes
# Genes: prefer adata.var['gene_ids'] or var_names; Seurat needs 2 columns in features.tsv
var = adata.var.copy()
if "gene_ids" in var.columns:
    gene_ids = var["gene_ids"].astype(str).values
else:
    gene_ids = adata.var_names.astype(str).values

gene_names = adata.var_names.astype(str).values

# Barcodes = cell IDs (obs_names)
barcodes = adata.obs_names.astype(str).values

# 5) Write 10x files: matrix.mtx.gz, features.tsv.gz, barcodes.tsv.gz
# IMPORTANT: 10x format expects matrix is genes x cells
# AnnData is cells x genes, so transpose!
mtx_path = os.path.join(OUT_DIR, "matrix.mtx")
mmwrite(mtx_path, X_counts.T)  # X_counts is cells x genes? we made X_counts from adata (cells x genes).
# Wait: X_counts currently derived from adata (cells x genes), but we transposed in mmwrite => genes x cells.

# gzip the matrix.mtx
with open(mtx_path, "rb") as f_in, gzip.open(mtx_path + ".gz", "wb") as f_out:
    f_out.writelines(f_in)
os.remove(mtx_path)

# features.tsv.gz : 2 columns (gene_id, gene_name) is standard
features_path = os.path.join(OUT_DIR, "features.tsv.gz")
with gzip.open(features_path, "wt") as f:
    for gid, gname in zip(gene_ids, gene_names):
        f.write(f"{gid}\t{gname}\n")

# barcodes.tsv.gz : 1 column
barcodes_path = os.path.join(OUT_DIR, "barcodes.tsv.gz")
with gzip.open(barcodes_path, "wt") as f:
    for bc in barcodes:
        f.write(f"{bc}\n")

print("Saved 10x RNA folder:", OUT_DIR)
print("Counts source used:", counts_source)
print("Files:", os.listdir(OUT_DIR))

# 6) Save metadata for Seurat (highly recommended)
meta = adata.obs.copy()
meta.insert(0, "cell_id", adata.obs_names.astype(str))

meta_path = os.path.join(OUT_DIR, "obs_metadata.csv.gz")
meta.to_csv(meta_path, index=False, compression="gzip")
print("Saved metadata:", meta_path)

# Optional: also save var metadata (gene annotations)
var_out = var.copy()
var_out.insert(0, "gene_name", adata.var_names.astype(str))
var_path = os.path.join(OUT_DIR, "var_metadata.csv.gz")
var_out.to_csv(var_path, index=False, compression="gzip")
print("Saved gene metadata:", var_path)


prparing R inviroment and importing the data

In [ ]:
%%R
options(repos = "https://cloud.r-project.org")

pkgs <- c("Seurat", "Matrix", "ggplot2")
new <- pkgs[!(pkgs %in% installed.packages()[,"Package"])]
if (length(new)) install.packages(new, Ncpus = 2)

library(Seurat)
library(Matrix)
library(ggplot2)

# Stability for big jobs
options(future.globals.maxSize = 12e9)
options(expressions = 5e5)

sessionInfo()

(as ‘lib’ is unspecified)






































































	‘/tmp/Rtmpgugh2v/downloaded_packages’



Attaching package: ‘SeuratObject’



    intersect, t





    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    R version 4.5.2 (2025-10-31)
Platform: x86_64-pc-linux-gnu
Running under: Ubuntu 22.04.4 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas-pthread/libblas.so.3 
LAPACK: /usr/lib/x86_64-linux-gnu/openblas-pthread/liblapack.so.3;  LAPACK version 3.10.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] tools     stats     graphics  grDevices utils     datasets  methods  
[8] base     

other

In [ ]:
%%R
tenx_dir <- "/content/drive/MyDrive/Datasets/GSE194122/seurat/RNA_10x"
meta_path <- file.path(tenx_dir, "obs_metadata.csv.gz")
base_out <- "/content/drive/MyDrive/Datasets/GSE194122/seurat"
dir.create(base_out, recursive = TRUE, showWarnings = FALSE)

# Read counts
counts <- Read10X(data.dir = tenx_dir)
if (is.list(counts)) {
  if ("Gene Expression" %in% names(counts)) {
    counts <- counts[["Gene Expression"]]
  } else if ("RNA" %in% names(counts)) {
    counts <- counts[["RNA"]]
  } else {
    counts <- counts[[1]]
  }
}

obj <- CreateSeuratObject(counts = counts, project = "GSE194122", min.cells = 0, min.features = 0)
rm(counts); gc()

# Read metadata (gz) with base R
meta <- read.csv(gzfile(meta_path), stringsAsFactors = FALSE)
stopifnot("cell_id" %in% colnames(meta))
rownames(meta) <- meta$cell_id

# Align and attach
common_cells <- intersect(colnames(obj), rownames(meta))
obj <- subset(obj, cells = common_cells)
obj <- AddMetaData(obj, metadata = meta[common_cells, , drop = FALSE])

message("Loaded Seurat object:"); print(obj)
message("Meta columns (first 30):"); print(head(colnames(obj@meta.data), 30))

In [ ]:
%%R
batch_key <- "batch"  # <- keep for this dataset
stopifnot(batch_key %in% colnames(obj@meta.data))
message("Using batch_key = ", batch_key, " | batches = ", length(unique(obj[[batch_key]][,1])))

# Helper: export UMAP + latent + plot + object
export_embeddings <- function(obj, outdir, latent_reduction, umap_reduction, batch_key) {
  dir.create(outdir, recursive = TRUE, showWarnings = FALSE)

  # UMAP embeddings
  umap <- as.data.frame(Embeddings(obj, reduction = umap_reduction))
  write.csv(umap, file = file.path(outdir, "umap_embeddings.csv"), quote = FALSE)

  # Latent embeddings
  lat <- as.data.frame(Embeddings(obj, reduction = latent_reduction))
  write.csv(lat, file = file.path(outdir, "latent_embeddings.csv"), quote = FALSE)

  # UMAP plot by batch
  p <- DimPlot(obj, reduction = umap_reduction, group.by = batch_key, pt.size = 0.1) +
    ggtitle(paste0("UMAP colored by ", batch_key, " | ", basename(outdir)))
  ggsave(filename = file.path(outdir, "umap_by_batch.png"),
         plot = p, width = 7, height = 5, dpi = 200)

  # Optional: UMAP by cell type if present
  if ("cell_type" %in% colnames(obj@meta.data)) {
    p2 <- DimPlot(obj, reduction = umap_reduction, group.by = "cell_type", pt.size = 0.1, label = FALSE) +
      ggtitle(paste0("UMAP colored by cell_type | ", basename(outdir)))
    ggsave(filename = file.path(outdir, "umap_by_cell_type.png"),
           plot = p2, width = 7, height = 5, dpi = 200)
  }

  # Save object
  saveRDS(obj, file = file.path(outdir, "seurat_object.rds"))
  message("✅ Exported to: ", outdir)
}

# Helper: safe checkpoint write
checkpoint <- function(x, path) {
  dir.create(dirname(path), recursive = TRUE, showWarnings = FALSE)
  saveRDS(x, file = path)
  message("💾 Saved checkpoint: ", path)
}

In [ ]:
%%R
if (!requireNamespace("future", quietly = TRUE)) install.packages("future", Ncpus = 2)
library(future)

# Keep your current plan (if any), just increase the cap
options(future.globals.maxSize = 30 * 1024^3)  # 30GB

V5

In [ ]:
%%R
out_v5 <- file.path(base_out, "v5")
dir.create(out_v5, recursive = TRUE, showWarnings = FALSE)

# If you already finished v5 before, you can reload:
v5_ckpt <- file.path(out_v5, "v5_integrated.rds")
if (file.exists(v5_ckpt)) {
  message("Loading existing v5 checkpoint...")
  v5 <- readRDS(v5_ckpt)
} else {
  v5 <- obj

  # Create layers split by batch (Seurat v5)
  v5[["RNA"]] <- split(v5[["RNA"]], f = v5[[batch_key]][,1])

  v5 <- NormalizeData(v5, verbose = FALSE)
  v5 <- FindVariableFeatures(v5, nfeatures = 3000, verbose = FALSE)
  v5 <- ScaleData(v5, verbose = FALSE)
  v5 <- RunPCA(v5, npcs = 30, verbose = FALSE)

  v5 <- IntegrateLayers(
    object = v5,
    method = RPCAIntegration,
    orig.reduction = "pca",
    new.reduction = "integrated.rpca",
    verbose = FALSE
  )

  v5 <- RunUMAP(v5, reduction = "integrated.rpca", dims = 1:30, reduction.name = "umap.v5", verbose = FALSE)

  checkpoint(v5, v5_ckpt)
}

export_embeddings(v5, out_v5, latent_reduction = "integrated.rpca", umap_reduction = "umap.v5", batch_key = batch_key)
gc()

V4

In [ ]:
%%R
out_v4 <- file.path(base_out, "v4")
dir.create(out_v4, recursive = TRUE, showWarnings = FALSE)

v4_final_ckpt <- file.path(out_v4, "v4_final.rds")
if (file.exists(v4_final_ckpt)) {
  message("Loading existing v4 checkpoint...")
  v4 <- readRDS(v4_final_ckpt)
} else {
  v4_anchors_ckpt <- file.path(out_v4, "v4_anchors.rds")
  v4_integrated_ckpt <- file.path(out_v4, "v4_integrated_before_dimred.rds")

  if (file.exists(v4_integrated_ckpt)) {
    message("Loading v4 integrated checkpoint...")
    v4 <- readRDS(v4_integrated_ckpt)
  } else {
    if (file.exists(v4_anchors_ckpt)) {
      message("Loading v4 anchors checkpoint...")
      anchors <- readRDS(v4_anchors_ckpt)
    } else {
      objs <- SplitObject(obj, split.by = batch_key)

      # Best-practice SCT
      objs <- lapply(objs, function(x) SCTransform(x, verbose = FALSE))

      features <- SelectIntegrationFeatures(objs, nfeatures = 3000)
      objs <- PrepSCTIntegration(objs, anchor.features = features)

      # RPCA requires PCA per batch object
      objs <- lapply(objs, function(x) RunPCA(x, npcs = 50, verbose = FALSE))

      anchors <- FindIntegrationAnchors(
        object.list = objs,
        normalization.method = "SCT",
        anchor.features = features,
        reduction = "rpca",
        dims = 1:50
      )

      checkpoint(anchors, v4_anchors_ckpt)
      rm(objs); gc()
    }

    v4 <- IntegrateData(anchorset = anchors, normalization.method = "SCT")
    checkpoint(v4, v4_integrated_ckpt)
    rm(anchors); gc()
  }

  DefaultAssay(v4) <- "integrated"
  v4 <- RunPCA(v4, npcs = 30, verbose = FALSE)
  v4 <- RunUMAP(v4, reduction = "pca", dims = 1:30, reduction.name = "umap.v4", verbose = FALSE)

  checkpoint(v4, v4_final_ckpt)
}

export_embeddings(v4, out_v4, latent_reduction = "pca", umap_reduction = "umap.v4", batch_key = batch_key)
gc()

V3

In [ ]:
%%R
out_v3 <- file.path(base_out, "v3")
dir.create(out_v3, recursive = TRUE, showWarnings = FALSE)

v3_final_ckpt <- file.path(out_v3, "v3_final.rds")
if (file.exists(v3_final_ckpt)) {
  message("Loading existing v3 checkpoint...")
  v3 <- readRDS(v3_final_ckpt)
} else {
  v3_anchors_ckpt <- file.path(out_v3, "v3_anchors.rds")
  v3_integrated_ckpt <- file.path(out_v3, "v3_integrated_before_dimred.rds")

  if (file.exists(v3_integrated_ckpt)) {
    message("Loading v3 integrated checkpoint...")
    v3 <- readRDS(v3_integrated_ckpt)
  } else {
    if (file.exists(v3_anchors_ckpt)) {
      message("Loading v3 anchors checkpoint...")
      anchors <- readRDS(v3_anchors_ckpt)
    } else {
      objs <- SplitObject(obj, split.by = batch_key)

      # Best-practice SCT
      objs <- lapply(objs, function(x) SCTransform(x, verbose = FALSE))

      features <- SelectIntegrationFeatures(objs, nfeatures = 3000)
      objs <- PrepSCTIntegration(objs, anchor.features = features)

      # v3-style anchors = default CCA reduction
      anchors <- FindIntegrationAnchors(
        object.list = objs,
        normalization.method = "SCT",
        anchor.features = features
      )

      checkpoint(anchors, v3_anchors_ckpt)
      rm(objs); gc()
    }

    v3 <- IntegrateData(anchorset = anchors, normalization.method = "SCT")
    checkpoint(v3, v3_integrated_ckpt)
    rm(anchors); gc()
  }

  DefaultAssay(v3) <- "integrated"
  v3 <- RunPCA(v3, npcs = 30, verbose = FALSE)
  v3 <- RunUMAP(v3, reduction = "pca", dims = 1:30, reduction.name = "umap.v3", verbose = FALSE)

  checkpoint(v3, v3_final_ckpt)
}

export_embeddings(v3, out_v3, latent_reduction = "pca", umap_reduction = "umap.v3", batch_key = batch_key)
gc()

Check the outputs

In [ ]:
import os, glob

base_out = "/content/drive/MyDrive/Datasets/GSE194122/seurat"
for v in ["v3","v4","v5"]:
    p = os.path.join(base_out, v)
    print("\n==", v, "==")
    print("exists:", os.path.exists(p))
    print("files:", [os.path.basename(x) for x in glob.glob(p+"/*")])